In [1]:
!pip install pyserial numpy sklearn


  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2950 sha256=5b567a42c5b5cb707b9533a3814a639856c8eca9ea1ccca52f8c02bcde8311d7
  Stored in directory: /home/might_dai_46/.cache/pip/wheels/de/6a/05/7cad3fe50c04cfcba9f13bf25766f6fa85764e4f76f5c4fd98
Successfully built sklearn


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import serial
import time
import concurrent.futures
import glob

# Load and preprocess data
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(url)
data.dropna(inplace=True)
data = data.drop(columns=["Name", "Ticket", "Cabin", "PassengerId"])
data["Sex"] = data["Sex"].map({"male": 0, "female": 1})
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})
data["FamilySize"] = data["SibSp"] + data["Parch"] + 1
data = data.drop(columns=["SibSp", "Parch"])

X = data.drop("Survived", axis=1)
y = data["Survived"].to_numpy()

# Perform PCA for dimensionality reduction
pca = PCA(n_components=6)
X = pca.fit_transform(X)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Establish serial communication with the Arduinos
arduino_ports = glob.glob('/dev/ttyACM*')
serial_connections = [serial.Serial(port, 9600, timeout=1) for port in arduino_ports]
time.sleep(10)  # Give some time for the connections to establish

# Other functions and procedures are the same as in the original code

# ...

# Establish serial communication with the Arduinos
arduino_ports =glob.glob('/dev/ttyACM*')
serial_connections = [serial.Serial(port, 9600, timeout=1) for port in arduino_ports]
time.sleep(10)  # Give some time for the connections to establish

def receive_weights(ser):
    while True:
        line = ser.readline().decode().strip()
        if line.startswith("weights,"):
            weights_str = line[len("weights,"):].split(',')
            weights = [float(w) for w in weights_str]
            return weights

def train_on_arduino_with_index(i, X_subset, y_subset, batch_size=200):
    ser = serial_connections[i]
    board_id = i + 1
    data = list(zip(y_subset, X_subset))
    
    epochs = 100  # Define epochs
    learning_rate = 0.01  # Define learning_rate

    print(f"Sending train command to Arduino {board_id}")  # Debug message
    ser.write(f"{board_id},train\n".encode())
    ser.write(f"{epochs},{learning_rate},{len(data)},{batch_size}\n".encode())

    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        for sample in batch:
            ser.write(f"{sample[0]},{','.join(map(str, sample[1]))}\n".encode())

    print(f"Waiting for weights from Arduino {board_id}")  # Debug message
    weights = receive_weights(ser)
    print(f"Received weights from Arduino {board_id}: {weights}")  # Debug message
    return weights

# Divide the training data into subsets for each Arduino
n_arduinos = len(serial_connections)
subsets = np.array_split(np.arange(len(X_train)), n_arduinos)
X_subsets = [X_train[idx] for idx in subsets]
y_subsets = [y_train[idx] for idx in subsets]
# Train on all Arduinos in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(train_on_arduino_with_index, i, X_subsets[i], y_subsets[i]) for i in range(n_arduinos)]
    all_weights = [f.result() for f in futures]

# Average the weights from all Arduinos
average_weights = np.mean(all_weights, axis=0)

print("Average weights:", average_weights)

# Test the model
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

y_pred = np.round(sigmoid(X_test @ average_weights))

accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the serial connections
for ser in serial_connections:
    ser.close()


Sending train command to Arduino 1
Sending train command to Arduino 2
Sending train command to Arduino 3
Sending train command to Arduino 4
Sending train command to Arduino 5


SerialException: write failed: [Errno 5] Input/output error